## **Task 1: Feature Engineering**



##### **Load & Prepare the Dataset**

In [ ]:
import numpy as np
import pandas as pd
from joblib import dump

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import os
os.listdir('/content')

In [ ]:
# Load training data
df = pd.read_csv(
    "train_FD001.txt",
    sep=r"\s+",
    header=None
) # Read the file and split columns wherever there are one or more spaces or tabs, and assume there is no header row.

In [ ]:
# Column names based on NASA documentation
cols = (
    ["engine_id", "cycle"] +
    [f"op_setting_{i}" for i in range(1, 4)] +
    [f"sensor_{i}" for i in range(1, 22)]
)

In [ ]:
df.columns = cols

In [ ]:
df.head()

##### **Create Remaining Useful Life (RUL)**

In [ ]:
# Max cycle per engine
max_cycle = df.groupby("engine_id")["cycle"].max()

In [ ]:
# Map max cycle
df["max_cycle"] = df["engine_id"].map(max_cycle)

In [ ]:
# Remaining Useful Life
df["RUL"] = df["max_cycle"] - df["cycle"]

##### **Create 24-Hour Failure Label (Classification Target)**

In [ ]:
FAILURE_WINDOW = 24  # 24 hours / cycles

In [ ]:
df["failure_next_24hrs"] = (df["RUL"] <= FAILURE_WINDOW).astype(int) # If Remaining Useful Life (RUL) ≤ 24 cycles, failure will happen within the next 24 hours

In [ ]:
df[['RUL','failure_next_24hrs']]

#### **Feature Engineering**

##### **A. Rolling Mean & Standard Deviation(Last 1, 6, 12 hours)**

In [ ]:
WINDOWS = [6, 12] # rolling time windows
SENSORS = [f"sensor_{i}" for i in range(1, 22)] # list of sensor columns

Why 1 is NOT included in WINDOWS = [6, 12]?
* The rolling mean of 1 value = the value itself
* So this feature is identical to the original sensor
* It adds no new pattern or trend
* Standard deviation needs at least 2 values
* With 1 value → result is NaN

In [ ]:
feature_dict = {} # Collect features in a dictionary which helps to generate features WITHOUT inserting into df

In [ ]:
for sensor in SENSORS:
    for w in WINDOWS:
        feature_dict[f"{sensor}_roll_mean_{w}"] = (
            df.groupby("engine_id")[sensor] # Each engine has its own independent life cycle. Rolling stats are calculated only within the same engine
              .rolling(window=w) # Look at the previous w time steps, including the current one.
              .mean() # it captures: Overall trend & Smooths noisy sensor data
              .reset_index(level=0, drop=True)# groupby().rolling() creates a MultiIndex. Pandas can’t assign it directly to the DataFrame .Removes the engine_id index level & Aligns values correctly with original rows
        )

        feature_dict[f"{sensor}_roll_std_{w}"] = (
            df.groupby("engine_id")[sensor]
              .rolling(window=w)
              .std() # it captures: Variability / instability & Sudden fluctuations often indicate degradation
              .reset_index(level=0, drop=True)
        )
# For every sensor and every time window, it creates rolling statistical features (mean and standard deviation) separately for each engine.
# This helps the ML model understand trends and variability in sensor behavior over time.

In [ ]:
rolling_features = pd.DataFrame(feature_dict)

In [ ]:
df = pd.concat([df, rolling_features], axis=1)

### **B. Exponential Moving Average (EMA)**

EMA gives more weight to recent sensor values.

In [ ]:
ema_features = {} # It helps to stores the generated EMA feature in a dictionary instead of directly adding it to df

In [ ]:
for sensor in SENSORS: # Loops through all sensor columns
    ema_features[f"{sensor}_ema_6"] = (
        df.groupby("engine_id")[sensor] # Groups data by engine. Ensures EMA is calculated per engine lifecycle
          .ewm(span=6, adjust=False) # Applies Exponential Weighted Moving Average. span=6 → recent 6 time steps get higher weight. adjust=False → uses recursive EMA formula (standard in ML & signal processing)
          .mean() # Computes the EMA values. Despite the name, EMA is not a simple average—it weights recent values more heavily.
          .reset_index(level=0, drop=True) # Removes the engine_id index created by groupby()
    )

    ema_features[f"{sensor}_ema_12"] = (
        df.groupby("engine_id")[sensor]
          .ewm(span=12, adjust=False)
          .mean()
          .reset_index(level=0, drop=True)
    )

This code creates Exponential Moving Average (EMA) features for each sensor, calculated separately for each engine, using two time windows:

* EMA(6) → short-term behavior

* EMA(12) → medium-term behavior

These features help the model detect early degradation patterns in sensor readings.

In [ ]:
ema_df = pd.DataFrame(ema_features)

In [ ]:
df = pd.concat([df, ema_df], axis=1)

### **C. Lag Features (t-1, t-2)**

Lag features capture temporal dependency.

This creates lag (historical) features for each sensor so the ML model can learn:

How past sensor values influence future failures

Lag features are essential in time-series prediction and prevent data leakage.

In [ ]:
LAGS = [1, 2] #Lag-1 → previous time step
# Lag-2 → two time steps ago

In [ ]:
lag_features = {}

In [ ]:
for sensor in SENSORS:
    for lag in LAGS:
        lag_features[f"{sensor}_lag_{lag}"] = (
            df.groupby("engine_id")[sensor]
              .shift(lag) # Shifts sensor values backward in time & Creates historical context
        )

In [ ]:
lag_df = pd.DataFrame(lag_features)

In [ ]:
df = pd.concat([df, lag_df], axis=1)

In [ ]:
df.columns

### **Handle Missing Values**


Rolling and lag features create NaN values at the beginning of each time series.

The maximum window or lag tells us how many initial rows must be dropped to remove all NaNs safely.

In [ ]:
MAX_WINDOW = max(max(WINDOWS), max(LAGS)) #finds the largest value among all rolling window sizes (WINDOWS) and lag steps (LAGS)

In [ ]:
MAX_WINDOW

In [ ]:
df = df[df.groupby("engine_id").cumcount() >= MAX_WINDOW].reset_index(drop=True)
#removes the first few rows of each engine where time-series features (rolling, lag, EMA) are not fully available.
#cumcount():It counts rows within each group, starting from 0, and increases by 1 for every new row in that group.

### **Select Final Feature Set**

In [ ]:
FEATURE_COLUMNS = [
    col for col in df.columns
    if "sensor_" in col and
    ("roll" in col or "ema" in col or "lag" in col)
]

In [ ]:
X = df[FEATURE_COLUMNS]
y = df["failure_next_24hrs"]

In [ ]:
X.isna().sum().sum()

### **Efficient Serialization Using joblib**


In [ ]:
dump(X, "X_features_FD001.joblib")# dump() serializes (converts) Python objects into binary files. Files are saved on disk with .joblib extension
dump(y, "y_labels_FD001.joblib")

In [ ]:
from joblib import load

In [ ]:
X = load("X_features_FD001.joblib") # Reads binary .joblib files. Reconstructs the original Python objects in memory
y = load("y_labels_FD001.joblib")

## **Task 2: Modeling**


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
#stratify=y: Ensures failures appear in both train & test. Mandatory for <1% imbalance

### **Baseline Model: Logistic Regression**


In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg_pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()), #Scales all features to the same range so Logistic Regression works properly

    ("smote", SMOTE(
        sampling_strategy="auto", #Generates synthetic samples for the minority class to balance the dataset.(applied only on training data, no data leakage).
        random_state=42
    )),

    ("model", LogisticRegression(
        max_iter=1000, #ensures convergence
        class_weight=None, #not needed because SMOTE balances classes
        n_jobs=-1 #uses all CPU cores for faster training
    ))
])

In [ ]:
logreg_pipeline.fit(X_train, y_train)

In [ ]:
#Evaluate Logistic Regression (PR-AUC)
from sklearn.metrics import average_precision_score, precision_recall_curve

y_probs_lr = logreg_pipeline.predict_proba(X_test)[:, 1]
pr_auc_lr = average_precision_score(y_test, y_probs_lr)
#calculates the probability of positive class predictions and evaluates the model using precision-recall performance.

print("Logistic Regression PR-AUC:", pr_auc_lr)

**Precision–Recall tradeoff (High Precision focus)**

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_probs_lr)

pr_df = pd.DataFrame({
    "precision": precision[:-1],
    "recall": recall[:-1],
    "threshold": thresholds
})

pr_df.sort_values("precision", ascending=False).head()

### **Production Model: XGBoost**

In [ ]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# Count class distribution
negative_samples = (y_train == 0).sum()
positive_samples = (y_train == 1).sum() #Counts how many non-failure (0) and failure (1) samples are in the training data.

scale_pos_weight = negative_samples / positive_samples #Computes how much more weight the model should give to the rare failure class

print("scale_pos_weight:", scale_pos_weight)

In [ ]:
xgb_production_pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()), #Scales numerical features so the model trains more stably

    ("model", XGBClassifier(
        objective="binary:logistic", #binary classification with probability output
        eval_metric="aucpr", #evaluates using Precision-Recall AUC (best for imbalance)
        scale_pos_weight=scale_pos_weight, #increases importance of rare failure class

        n_estimators=400, #number of boosting trees
        max_depth=6, #controls model complexity
        learning_rate=0.05, #slow, stable learning
        subsample=0.8,
        colsample_bytree=0.8, #reduce overfitting

        n_jobs=-1, #use all CPU cores
        random_state=42 #reproducible results
    ))
])

In [ ]:
xgb_production_pipeline.fit(X_train, y_train)

In [ ]:
#Evaluate XGBoost
y_pred_proba = xgb_production_pipeline.predict_proba(X_test)[:, 1] #[:, 1] selects the probability of the positive (failure) class

pr_auc = average_precision_score(y_test, y_pred_proba)
print("PR-AUC:", pr_auc)

Precision–Recall Curve

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)

plt.figure(figsize=(6, 4))
plt.plot(recall, precision, label=f"PR-AUC = {pr_auc:.4f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.grid(True)
plt.show()

**Threshold Selection (High Precision Strategy)**

selects the classification threshold that achieves ≥95% precision while maximizing recall

In [ ]:
target_precision = 0.95

thresholds = thresholds[:len(precision)-1] #Aligns the number of thresholds with precision/recall values (PR curves return one extra precision point).

valid_idx = np.where(precision[:-1] >= target_precision)[0] #Finds indices where precision is at least 0.95.

best_idx = valid_idx[np.argmax(recall[valid_idx])] #Among those valid points, selects the one with the highest recall.

best_threshold = thresholds[best_idx] #Gets the decision threshold corresponding to that best point.

print("Selected threshold:", best_threshold)
print("Precision:", precision[best_idx])
print("Recall:", recall[best_idx])

In [ ]:
#converts predicted probabilities into final binary class predictions using the custom threshold instead of the default 0.5.
y_pred_custom = (y_pred_proba >= best_threshold).astype(int)

In [ ]:
#Final Evaluation Metrics
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_custom, digits=4))

### **Hyperparameter Tuning via GridSearchCV**

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer

In [ ]:
# Define Pipeline
xgb_pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("xgb", XGBClassifier(
        objective="binary:logistic",
        eval_metric="aucpr",
        scale_pos_weight=scale_pos_weight,
        n_jobs=-1,
        random_state=42
    ))
])

In [ ]:
# Hyperparameter Grid
param_grid = {
    "xgb__n_estimators": [200, 400],
    "xgb__max_depth": [4, 6],
    "xgb__learning_rate": [0.03, 0.05],
    "xgb__subsample": [0.8, 1.0],
    "xgb__colsample_bytree": [0.8, 1.0]
}

In [ ]:
# PR-AUC Scorer:This scorer makes GridSearchCV evaluate models using PR-AUC
pr_auc_scorer = make_scorer(  #Creates a custom scoring function for model evaluation
    average_precision_score,
    needs_proba=True #tells GridSearchCV to use predicted probabilities, not class labels
)

In [ ]:
# GridSearchCV Setup
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) #Splits data into 3 folds.Stratified → keeps class imbalance ratio the same in each fold. shuffle=True → randomizes samples

grid_search = GridSearchCV(
    estimator=xgb_pipeline,
    param_grid=param_grid, #Tries all parameter combinations in param_grid
    scoring=pr_auc_scorer,
    cv=cv, #Uses stratified cross-validation
    n_jobs=-1,
    verbose=2
)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
#Best Parameters
print("Best Parameters:")
print(grid_search.best_params_)

In [ ]:
#Final Model (GridSearch Best)
best_xgb_grid = grid_search.best_estimator_

In [ ]:
#Evaluate on Test Data
y_pred_proba = best_xgb_grid.predict_proba(X_test)[:, 1]

pr_auc = average_precision_score(y_test, y_pred_proba)
print("Test PR-AUC:", pr_auc)